In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import numpy as np
import pandas as pd
from scipy.io import loadmat
from sklearn.preprocessing import StandardScaler
from sklearn import decomposition

In [ ]:
def generate_pca_output(data_path, subject, out_path):
    read_dir = data_path + '/' + subject
    filenames = sorted(os.listdir(read_dir))
    x = np.zeros((len(filenames),3000000), dtype='float32')
  
    for filename in filenames:
        matfile = loadmat(read_dir + '/' + filename, squeeze_me=True)
        key_list = matfile.keys()
        matching = [s for s in key_list if "segment" in s]
        segment_name = matching[0]
        data = matfile[segment_name].flat[0][0]
        chnls = matfile[segment_name].flat[0][3]

        df = pd.DataFrame(data.T, columns = chnls)

        x_std = StandardScaler().fit_transform(df)
        pca = decomposition.PCA(n_components=1)
        pca_res = pca.fit_transform(x_std)
        np.savetxt(out_path + '/' + filename + '_pca.csv', pca_res, delimiter=",",fmt='%f')

In [ ]:
output_folder = '/kaggle/working/Patient_1'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
generate_pca_output('/kaggle/input/seizure-prediction/Patient_1', 'Patient_1', output_folder)

In [ ]:
!pip install pyts

In [ ]:
#!pip uninstall --yes scikit-learn

In [ ]:
#    !pip install scikit-learn=='0.22.1'

In [ ]:
from pyts.classification import TimeSeriesForest
import os

def classify(data_path, subject, out_path):
    #Run PCA on input files, save the first principal component
#     if not os.path.exists(out_path + '/' + subject):
#         os.makedirs(out_path + '/' + subject )
#     generate_pca_output(data_path, subject, out_path + '/' + subject)
    
    #Read the input files
    train_data = load_train_data(out_path, subject)
    test_data = load_test_data(out_path, subject)
    
    #Train the random forest algorithm
    clf = TimeSeriesForest(n_estimators=500, n_windows=500, min_window_size=30)
    clf.fit(train_data['x'],train_data['y'])
    print('Training done!!!')
    clf_res = clf.predict(test_data['x'])
    results = pd.DataFrame(list(zip(test_data['filename'], clf_res)), columns =['clip', 'preictal'])
    results.to_csv(out_path + '/' + subject + '.csv')
    return clf_res

In [ ]:
import shutil
shutil.make_archive('Patient_1.zip', 'zip', '/kaggle/working/Patient_1')

import os -> os.chdir(r'/kaggle/working')
from IPython.display import FileLink -> FileLink(r'*name of file*')



In [ ]:
def load_pca_data(data_path, subject, test_data=False):
  read_dir = data_path + '/' + subject
  filenames = sorted(os.listdir(read_dir))
  train_filenames = []
  for filename in filenames:
    if test_data:
      if 'test' in filename:
        train_filenames.append(filename)
    else:
      if 'test' not in filename:
        train_filenames.append(filename)

  df = pd.read_csv(read_dir + '/' + train_filenames[0], header=None)
  print(df.shape)

  n = len(train_filenames)
  x = np.zeros((n,df.shape[0]), dtype='float32')
  y = np.zeros(n, dtype='int8')
  filename = np.zeros(n, dtype='object')
  print(df[0])
    
  idx = 0
  for i, fname in enumerate(train_filenames):
    df = pd.read_csv(read_dir + '/' + train_filenames[0], header=None)
    x[i] = df[0]
    y[i] = 1 if 'preictal' in fname else 0
    filename[i] = fname[:-8]

  return {'x': x, 'y': y, 'filename': filename}

In [ ]:
def load_train_data(data_path, subject):
  return load_pca_data(data_path, subject)

def load_test_data(data_path, subject):
  return load_pca_data(data_path, subject, True)

In [ ]:
classify("/kaggle/input/seizure-prediction/Patient_1", "Patient_1", "/kaggle/working")

In [ ]:
import os 
os.chdir(r'/kaggle/working')
from IPython.display import FileLink 
FileLink(r'Patient_1.zip.zip')

